#Context

Company X is promoting a campaign where users who makes payment by scanning QR code at merchants' store will be rewarded with Rebate.

Rebate is determined at 30% of transaction value, capped at $2 per txn.

It is reported that some users are colluding with merchants to abuse this promotion campaign. Our job is to identify the potential abusers, both users and merchants.

# Library import

In [275]:
# Data manipulation
import numpy as np
import pandas as pd
import datetime
from datetime import timedelta

# Options for pandas
from IPython.core.display import display, HTML
pd.options.display.max_columns = None
display(HTML("<style>.container { width:85% !important; }</style>"))
pd.options.display.float_format = '{:,}'.format

# Visualizations
import plotly.express as px
from plotly import graph_objects as go
import matplotlib.pyplot as plt
%matplotlib inline

# ML
from sklearn.datasets.samples_generator import (make_blobs,
                                                make_circles,
                                                make_moons)
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn import decomposition
from sklearn import datasets

#zen mode
import warnings
warnings.filterwarnings("ignore")

##Helper functions

In [0]:
# Styling
def highlight_cols(s):
    color = 'red'
    return 'background-color: %s' % color

def short_timedelta_highlight(s):    
  '''highlight the timedelta is less than 15 mins yellow.'''
  is_short = s<timedelta(minutes=30)
  return ['background-color: yellow' if v else '' for v in is_short]

In [0]:
def time_variance(sorted_time_column):
  '''This function takes a sorted column of datetime values, and return the minimum timedelta among them'''
  if sorted_time_column.shape[0] < 3:
    return None
  else:
    time_column_A = sorted_time_column[:-1].reset_index(drop=True)
    time_column_B = sorted_time_column[1:].reset_index(drop=True)
    time_diff = time_column_A - time_column_B
    return time_diff.min()

In [0]:
def visualise_cluster(orders):
  '''This function takes in a dataframe of orders, clusters it by txn_time and gmv, and returns the visualisation for the clusters'''
  df = orders.reset_index(drop=True)
  df_cluster = df[['txn_time', 'gmv']]
  df_cluster['txn_time'] = df_cluster['txn_time'].astype(int) / 10**9

  # Scale gmv columns
  sc = StandardScaler()
  df_cluster = pd.DataFrame(sc.fit_transform(df_cluster), columns=['txn_time', 'gmv'], index = df_cluster.index)

  # fit KMeans model
  km_scores= []
  km_silhouette = []

  for i in range(2,12):
    km = KMeans(n_clusters=i, random_state=0).fit(df_cluster)

    preds = km.predict(df_cluster)
    km_scores.append(-km.score(df_cluster))
    
    silhouette = silhouette_score(df_cluster,preds)
    km_silhouette.append(silhouette)

  # display cluster scorings
  from plotly.subplots import make_subplots
  fig = make_subplots(rows=1, cols=2)
  fig.add_trace(
    go.Scatter(x=list(range(2,12)), y=km_scores,name = 'kmeans score'),
    row=1, col=1)
  fig.add_trace(
    go.Scatter(x=list(range(2,12)), y=km_silhouette, name = 'silhouette score'),
    row=1, col=2)

  fig.update_layout(height=400, width=1000, title_text="Parameter scoring")
  fig.show()

  # Visualising clusters
  no_of_clusters = np.argmax(km_silhouette)+2
  model = KMeans(n_clusters=no_of_clusters, random_state=0)
  cluster = pd.DataFrame(model.fit_predict(df_cluster)).astype('category')
  cluster.columns = ['Cluster']
  df = df.merge(cluster, left_index=True, right_index=True)
  fig = px.scatter(df,
            x='txn_time',
            y='gmv',
            color='Cluster')
  fig.show()

  return None

# Solution

In [279]:
from google.colab import drive
drive.mount('/content/gdrive')

Orders = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Rebate Fraud detection/Rebate Fraud detection.csv')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Cleansing & EDA

In [280]:
Orders.head()

,txn_time,txn_date,order_id,uid,shop_id,gmv,rebate
0,10/30/2015 13:37,10/30/2015,8867,161617,11709,7.0,2.0
1,10/30/2015 20:26,10/30/2015,119519,867365,120249,8.0,2.0
2,10/30/2015 12:22,10/30/2015,986559,401483,120549,12.5,2.0
3,10/30/2015 9:46,10/30/2015,942099,213258,120705,8.2,2.0
4,10/30/2015 21:56,10/30/2015,139417,3391,121353,26.0,2.0


In [281]:
Orders.describe(include='all')

,txn_time,txn_date,order_id,uid,shop_id,gmv,rebate
count,4611,4611,"4,611.0","4,611.0","4,611.0","4,611.0","4,611.0"
unique,2521,4,nan,nan,nan,nan,nan
top,11/2/2015 13:05,11/2/2015,nan,nan,nan,nan,nan
freq,8,1529,nan,nan,nan,nan,nan
mean,NaN,NaN,"530,295.0561700282","420,764.422684884","131,795.8366948601",13.898523096942098,1.6813597918022125
std,NaN,NaN,"274,164.47839037824","319,435.26185221475","108,082.19736593976",19.39903931831104,0.6520429723315134
min,NaN,NaN,286.0,171.0,"1,281.0",0.0,0.0
25%,NaN,NaN,"309,398.5","100,364.0","67,365.0",7.0,1.95
50%,NaN,NaN,"570,966.0","395,464.0","100,845.0",8.49,2.0
75%,NaN,NaN,"768,706.0","698,105.5","130,125.0",13.9,2.0


In [0]:
Orders = Orders[Orders['gmv']>0]

In [0]:
Orders.txn_time = pd.to_datetime(Orders.txn_time)

In [285]:
px.histogram(Orders, x='gmv')

##Fraud detection - structured approach

Fraud indicators:
1. An user makes an unreasonably high number of transactions
2. A pair of shop owner - user makes an unreasonably high number of transactions
3. The rebate is at maximum, i.e. $2
4. Transactions are frequent within a short period of time
5. Rebate sum by user is maximised

### Indicator 1: An user makes an unreasonably high number of transactions

In [286]:
# Order count by uid
df = pd.DataFrame(Orders.groupby('uid').agg(['sum', 'count']))['rebate']
df.columns = ['Rebate_sum_by_user','Order_count_by_user']
px.histogram(df, x='Order_count_by_user')

In [0]:
Orders_extra = pd.merge(Orders, df, left_on='uid', right_index=True, how = 'left')

In [0]:
indicator_1 = Orders_extra['Order_count_by_user']>=10

### Indicator 2: A pair of shop owner - user makes an unreasonably high number of transactions

In [289]:
# Order count by shop-user pair
df = pd.DataFrame(Orders.groupby(['uid', 'shop_id']).agg(['sum', 'count']))['rebate']
df.columns = ['Rebate_sum_by_shop&user','Order_count_by_shop&user']
px.histogram(df, x='Order_count_by_shop&user')

In [0]:
Orders_extra = pd.merge(Orders_extra, df, left_on=['uid', 'shop_id'], right_index=True)

In [0]:
indicator_2 = Orders_extra['Order_count_by_shop&user']>=3

### Indicator_3: The rebate is at maximum, i.e. near $2

In [292]:
px.histogram(Orders_extra, x='rebate')

Since most rebate are $2, i.e. most users take full advantage of the rebate, this indicator is not effective. We shall drop this indicator.

###Indicator_4: Transactions are frequent within a short period of time

In [293]:
Orders_extra.sort_values(by=['txn_time'], ascending=False, inplace=True)

df = Orders_extra.groupby(by='uid')[['txn_time', 'gmv', 'rebate', 'order_id']]              \
  .agg({'txn_time': time_variance, 'gmv': 'mean', 'rebate': 'mean', 'order_id': 'count'})   \
  .dropna()                                                                                 \
  .reset_index()                                                                            \
  .sort_values('txn_time')
df.columns = ['uid', 'time_variance', 'average gmv', 'average rebate', 'count of order']
df.head(10).style.apply(short_timedelta_highlight, subset=['time_variance'])

,uid,time_variance,average gmv,average rebate,count of order
0,1018,0 days 00:00:00,4.750000,1.000000,4
127,308120,0 days 00:00:00,4.500000,1.000000,4
125,297464,0 days 00:00:00,4.500000,1.000000,4
122,285462,0 days 00:00:00,4.500000,1.000000,4
120,282727,0 days 00:00:00,5.500000,1.142857,7
258,839064,0 days 00:00:00,4.500000,1.000000,4
115,265578,0 days 00:00:00,10.000000,2.000000,3
113,263635,0 days 00:00:00,8.010000,2.000000,4
112,259847,0 days 00:00:00,6.700000,1.333333,6
261,849561,0 days 00:00:00,4.833333,1.000000,6


In [294]:
fig = go.Figure()
trace = go.Histogram(x=df['time_variance'].astype('timedelta64[m]'),
                     xbins=dict(
                     start=0,
                     size=20),
                     autobinx=False
                     )
fig.add_trace(trace)

In [0]:
indicator_4_uid = df.loc[df.time_variance <= timedelta(minutes=20), 'uid']
indicator_4 = Orders_extra['uid'].isin(indicator_4_uid)

### Indicator 5: Rebate sum by user is maximised


In [296]:
px.histogram(Orders_extra,
             x = 'Rebate_sum_by_user')

In [0]:
indicator_5 = Orders_extra['Rebate_sum_by_user']==Orders_extra['Rebate_sum_by_user'].max()

### Combine all indicators

In [0]:
for indicator in ['indicator_1', 'indicator_2', 'indicator_4', 'indicator_5']:
  Orders_extra[indicator] = eval(indicator)
Orders_extra['Number of flags'] = Orders_extra.indicator_1*1 + Orders_extra.indicator_2*1 \
                                 + Orders_extra.indicator_4*1 + Orders_extra.indicator_5*1

In [0]:
# Suspected orders are those with 3 flags or higher
Suspected_orders = Orders_extra[Orders_extra['Number of flags']>=3]

In [300]:
Suspected_uid = sorted(Suspected_orders['uid'].unique().tolist())
Suspected_shop_id = sorted(Suspected_orders['shop_id'].unique().tolist())
print(f' {len(Suspected_uid)} Suspected uid: {Suspected_uid}')
print(f' {len(Suspected_shop_id)} Suspected uid: {Suspected_shop_id}')

 7 Suspected uid: [23942, 366427, 653121, 653495, 668004, 682428, 868549]
 13 Suspected uid: [51753, 57357, 69081, 69345, 71121, 81729, 82557, 82653, 85821, 90153, 91161, 91485, 93369]


In [301]:
visualise_cluster(Suspected_orders)

## Fraud detection - naive approach

In [306]:
Suspected_uid = Orders_extra.loc[indicator_1, 'uid'].drop_duplicates()

Suspected_uid = sorted(Suspected_orders['uid'].unique().tolist())
print(f' {len(Suspected_uid)} Suspected uid: {Suspected_uid}')

 7 Suspected uid: [23942, 366427, 653121, 653495, 668004, 682428, 868549]


In [303]:
# suspected shopes are shops that makes more than 3 transactions with the suspected users
sr = Orders_extra[indicator_1].groupby('shop_id')['uid'].count()
Suspected_shop_id = sr[sr>3].index
Suspected_shop_id = sorted(Suspected_orders['shop_id'].unique().tolist())
print(f' {len(Suspected_shop_id)} Suspected uid: {Suspected_shop_id}')

[51753, 57357, 69081, 69345, 71121, 81729, 82557, 82653, 85821, 91485]


In [0]:
# Suspected_orders are orders that involve both the suspected shops and suspected users
Suspected_orders = Orders_extra[Orders_extra['uid'].isin(Suspected_uid) & 
                                Orders_extra['shop_id'].isin(Suspected_shop_id)]\
                                .sort_values('txn_time')

In [305]:
visualise_cluster(Suspected_orders)